In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
import string
import copy

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import itertools
import copy
import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL, DfToSqlMap
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from OutageDAQ import OutageDAQ, OutageDAQOtBL
from OutageDAQ import OutageDataInfo as ODI
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
from CustomJSON import CustomEncoder, CustomWriter
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSingleSlicer as DFSingleSlicer

# -----------------------------------------------------------------------------------------------
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# -----------------------------------------------------------------------------------------------

# Will have two methods for building.  
# One using a supplied df_outage with (or, I suppose, without) accompanying meter premise
# One building from ground up, given date range

In [ ]:
#----------------------------------------------------------------------------------------------------
# VARIABLES TO BE SET BY USER!
#----------------------------------------------------------------------------------------------------
# Unless absolutely certain df_mp in csv has all necessary data, use read_dfs_from_file=False
save_dfs_to_file   = True
read_dfs_from_file = False
save_end_events    = True

#-------------------------
# run_date is used to collect all results from a given acquisiton run together.
# As such, run_date should be set to the first date of the acquisition run, and
#   SHOULD NOT be changed for each individual date in a run (which typically lasts
#   over the course of days/weeks)
run_date = '20240906' # Date of data acquisition

#-------------------------
date_0 = '2023-04-01' # Lower limit for end events
date_1 = '2024-08-31' # Upper limit for end events

#-------------------------
# dataset = 'outg'
dataset = 'otbl'
# dataset = 'prbl'

#-------------------------
min_window_width = pd.Timedelta('31 days')
buffer_time_left = pd.Timedelta('1 days')
buffer_time_rght = pd.Timedelta('31 days')

#-------------------------
run_using_slim = False


#--------------------------------------------------
# If df_mp is read from csv, it will typically contain an outg_rec_nb column
#   and entries which are duplicates except for outg_rec_nb
# For this process to work correctly, these duplicates must be removed.
df_mp_outg_rec_nb_col = 'OUTG_REC_NB'

#-------------------------
groupby_col = 'trsf_pole_nb'
# groupby_col = 'PREMISE_NB'
assert(groupby_col in ['trsf_pole_nb', 'PREMISE_NB'])

#-------------------------
pd_selection_stategy = 'all'
# search_window_strategy = 'centered'
# search_window_strategy = pd.Timedelta('1 day')
search_window_strategy = 'all_subwindows'

#--------------------------------------------------
# NOTE: below, states and opcos should be consistent!
#       i.e., e.g., if states='OH', then opcos should be 'oh' (or None, I suppose)
#-------------------------
# states used to 
#   (1) find transformers which suffered at least one outage from DOVS
#   (2) find all transformers from MeterPremise
# states can be:
#   - a single string, e.g. 'OH'
#   - a list of strings, e.g., ['OH', 'WV']
#   - None
# NOTE: states tend to be upper-case!
states=['OH']

#-------------------------
# opcos used with AMIEndEvents to
#  (1) find the premise numbers which recorded an event between date_0 and date_1.
#  (2) selection/acquisiton of end_device_events
# opcos can be:
#   - a single string, e.g. 'oh'
#   - a list of strings, e.g., ['oh', 'tx']
#   - None
# NOTE: opcos tend to be lower-case!
# NOTE: Acceptable opcos appear to be: ['ap', 'im', 'oh', 'pso', 'swp', 'tx']
opcos='oh'

#-------------------------
# cities = None
cities = ['COLUMBUS']

#--------------------------------------------------
trsf_pole_nbs_to_ignore = [' ', 'TRANSMISSION', 'PRIMARY', 'NETWORK']

# TODO!!!!!!!!!!!!!!!!!!
single_zip_xfmrs_only = True

In [ ]:
# # UNCOMMENT AFTER DONE CHECKING EVS_SUM
# ############### NEW
# outg_daq = OutageDAQOtBL(
#     run_date                = run_date, 
#     date_0                  = date_0, 
#     date_1                  = date_1, 
#     collect_evs_sum_vw      = False,  
#     save_sub_dir            = ODI.get_subdir(dataset), 
#     groupby_col             = groupby_col, 
#     min_window_width        = min_window_width, 
#     buffer_time_left        = buffer_time_left, 
#     buffer_time_rght        = buffer_time_rght, 
#     pd_selection_stategy    = pd_selection_stategy, 
#     search_window_strategy  = search_window_strategy, 
#     states                  = states, 
#     opcos                   = opcos, 
#     cities                  = cities, 
#     single_zip_xfmrs_only   = single_zip_xfmrs_only, 
#     save_end_events         = save_end_events, 
#     save_dfs_to_file        = save_dfs_to_file, 
#     read_dfs_from_file      = read_dfs_from_file, 
#     base_dir                = os.path.join(
#         Utilities.get_local_data_dir(), 
#         r'dovs_and_end_events_data'
#     ), 
#     run_using_slim          = run_using_slim 
# )

In [ ]:
# # UNCOMMENT AFTER DONE CHECKING EVS_SUM
# #--------------------------------------------------
# # OUTAGES
# #--------------------------------------------------
# outg_daq.build_or_load_df_outage_OG(verbose=True)

# #--------------------------------------------------
# # Find all outages between date_0 and date_1 for PNs in df_outage
# #   Need to find all PNs, which consist not only of those directly from df_outage, 
# #   but also those not in df_outage who were connected to transformers having entries in df_outage
# #--------------------------------------------------
# outg_daq.build_or_load_mp_df_xfmrs(verbose=True)

# #--------------------------------------------------
# # Build zip code information for transformers, 
# # and possibly restrict data to single zipcode transformers only
# #--------------------------------------------------
# outg_daq.build_or_load_trsf_pole_zips_info(verbose=True)

# #--------------------------------------------------
# # Now, for all premise numbers in df_outage, we want to find all outages suffered between date_0 and date_1
# # There are two methods for achieving this, using either method='query_pns_only' or method='query_all' in find_all_outages_for_pns
# # NOTE: It is possible that all needed info already contained in df_outage! 
# #       But that is reliant upon the user having everything set up perfectly....so safest just to build...
# #--------------------------------------------------
# outg_daq.build_or_load_all_outages_df(
#     verbose               = True, 
#     n_update              = 10, 
#     batch_size            = 1000, 
#     method                = 'decide_at_runtime'
# )

# #--------------------------------------------------
# # Find the clean windows for each group and build df_no_outage
# #--------------------------------------------------
# outg_daq.build_or_load_df_no_outage(verbose = True)

# #--------------------------------------------------
# # Convert df_no_outage to consolidated (slim) verion if run_using_slim
# #--------------------------------------------------
# if run_using_slim:
#     outg_daq.build_or_load_df_no_outage_slim(verbose = True)

# #--------------------------------------------------
# # Collect Events
# #--------------------------------------------------
# outg_daq.collect_events(
#     batch_size = None, 
#     verbose    = True, 
#     n_update   = 1
# )

# CHECKING EVS_SUM METHOD VS ORIGINAL

In [ ]:
############### NEW
outg_daq = OutageDAQOtBL(
    run_date                = run_date, 
    date_0                  = date_0, 
    date_1                  = date_1, 
    collect_evs_sum_vw      = False,  
    save_sub_dir            = ODI.get_subdir(dataset), 
    groupby_col             = groupby_col, 
    min_window_width        = min_window_width, 
    buffer_time_left        = buffer_time_left, 
    buffer_time_rght        = buffer_time_rght, 
    pd_selection_stategy    = pd_selection_stategy, 
    search_window_strategy  = search_window_strategy, 
    states                  = states, 
    opcos                   = opcos, 
    cities                  = cities, 
    single_zip_xfmrs_only   = single_zip_xfmrs_only, 
    save_end_events         = save_end_events, 
    save_dfs_to_file        = save_dfs_to_file, 
    read_dfs_from_file      = read_dfs_from_file, 
    base_dir                = os.path.join(
        Utilities.get_local_data_dir(), 
        r'dovs_and_end_events_data'
    ), 
    run_using_slim          = run_using_slim 
)

In [ ]:
#--------------------------------------------------
# OUTAGES
#--------------------------------------------------
outg_daq.build_or_load_df_outage_OG(verbose=True)

#--------------------------------------------------
# Find all outages between date_0 and date_1 for PNs in df_outage
#   Need to find all PNs, which consist not only of those directly from df_outage, 
#   but also those not in df_outage who were connected to transformers having entries in df_outage
#--------------------------------------------------
outg_daq.build_or_load_mp_df_xfmrs(verbose=True)

#--------------------------------------------------
# Build zip code information for transformers, 
# and possibly restrict data to single zipcode transformers only
#--------------------------------------------------
outg_daq.build_or_load_trsf_pole_zips_info(verbose=True)

#--------------------------------------------------
# Now, for all premise numbers in df_outage, we want to find all outages suffered between date_0 and date_1
# There are two methods for achieving this, using either method='query_pns_only' or method='query_all' in find_all_outages_for_pns
# NOTE: It is possible that all needed info already contained in df_outage! 
#       But that is reliant upon the user having everything set up perfectly....so safest just to build...
#--------------------------------------------------
outg_daq.build_or_load_all_outages_df(
    verbose               = True, 
    n_update              = 10, 
    batch_size            = 1000, 
    method                = 'decide_at_runtime'
)

#--------------------------------------------------
# Find the clean windows for each group and build df_no_outage
#--------------------------------------------------
outg_daq.build_or_load_df_no_outage(verbose = True)

#--------------------------------------------------
# Convert df_no_outage to consolidated (slim) verion if run_using_slim
#--------------------------------------------------
if run_using_slim:
    outg_daq.build_or_load_df_no_outage_slim(verbose = True)


In [ ]:
# Chop down DF size for quick run comp
if run_using_slim:
    outg_daq.df_no_outage_slim = outg_daq.df_no_outage_slim.iloc[:1000]
else:
    outg_daq.df_no_outage = outg_daq.df_no_outage.sort_values(by=['no_outg_rec_nb', 'trsf_pole_nb', 'prem_nb', 't_search_min'], ignore_index=True).iloc[:10000]

In [ ]:
df_1 = outg_daq.df_no_outage.copy()

In [ ]:
#--------------------------------------------------
# Collect Events
#--------------------------------------------------
outg_daq.collect_events(
    batch_size = None, 
    verbose    = True, 
    n_update   = 1
)

In [ ]:
############### NEWEST
outg_daq = OutageDAQOtBL(
    run_date                = run_date, 
    date_0                  = date_0, 
    date_1                  = date_1, 
    collect_evs_sum_vw      = True,  
    save_sub_dir            = ODI.get_subdir(dataset), 
    groupby_col             = groupby_col, 
    min_window_width        = min_window_width, 
    buffer_time_left        = buffer_time_left, 
    buffer_time_rght        = buffer_time_rght, 
    pd_selection_stategy    = pd_selection_stategy, 
    search_window_strategy  = search_window_strategy, 
    states                  = states, 
    opcos                   = opcos, 
    cities                  = cities, 
    single_zip_xfmrs_only   = single_zip_xfmrs_only, 
    save_end_events         = save_end_events, 
    save_dfs_to_file        = save_dfs_to_file, 
    read_dfs_from_file      = read_dfs_from_file, 
    base_dir                = os.path.join(
        Utilities.get_local_data_dir(), 
        r'dovs_and_end_events_data'
    ), 
    run_using_slim          = run_using_slim 
)

In [ ]:
#--------------------------------------------------
# OUTAGES
#--------------------------------------------------
outg_daq.build_or_load_df_outage_OG(verbose=True)

#--------------------------------------------------
# Find all outages between date_0 and date_1 for PNs in df_outage
#   Need to find all PNs, which consist not only of those directly from df_outage, 
#   but also those not in df_outage who were connected to transformers having entries in df_outage
#--------------------------------------------------
outg_daq.build_or_load_mp_df_xfmrs(verbose=True)

#--------------------------------------------------
# Build zip code information for transformers, 
# and possibly restrict data to single zipcode transformers only
#--------------------------------------------------
outg_daq.build_or_load_trsf_pole_zips_info(verbose=True)

#--------------------------------------------------
# Now, for all premise numbers in df_outage, we want to find all outages suffered between date_0 and date_1
# There are two methods for achieving this, using either method='query_pns_only' or method='query_all' in find_all_outages_for_pns
# NOTE: It is possible that all needed info already contained in df_outage! 
#       But that is reliant upon the user having everything set up perfectly....so safest just to build...
#--------------------------------------------------
outg_daq.build_or_load_all_outages_df(
    verbose               = True, 
    n_update              = 10, 
    batch_size            = 1000, 
    method                = 'decide_at_runtime'
)

#--------------------------------------------------
# Find the clean windows for each group and build df_no_outage
#--------------------------------------------------
outg_daq.build_or_load_df_no_outage(verbose = True)

#--------------------------------------------------
# Convert df_no_outage to consolidated (slim) verion if run_using_slim
#--------------------------------------------------
if run_using_slim:
    outg_daq.build_or_load_df_no_outage_slim(verbose = True)


In [ ]:
# Chop down DF size for quick run comp
if run_using_slim:
    outg_daq.df_no_outage_slim = outg_daq.df_no_outage_slim.iloc[:1000]
else:
    outg_daq.df_no_outage = outg_daq.df_no_outage.sort_values(by=['no_outg_rec_nb', 'trsf_pole_nb', 'prem_nb', 't_search_min'], ignore_index=True).iloc[:10000]

In [ ]:
outg_daq.df_no_outage.equals(df_1)

In [ ]:
#--------------------------------------------------
# Collect Events
#--------------------------------------------------
outg_daq.collect_events(
    batch_size = None, 
    verbose    = True, 
    n_update   = 1
)

In [ ]:
assert(0)

# END CHECKING EVS_SUM METHOD VS ORIGINAL

# OLD DEV

### OutagesDAQ constructor

In [ ]:
cities

In [ ]:
#----------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------
# DFs will be saved in save_dir_base
# Collection of end events files will be saved in os.path.join(save_dir_base, 'EndEvents')
save_dir_base = os.path.join(
    Utilities.get_local_data_dir(), 
    r'dovs_and_end_events_data', 
    run_date, 
    f"{date_0.replace('-','')}_{date_1.replace('-','')}", 
    'NoOutgs_OLD'
)
#-------------------------
end_events_save_args = dict(
    save_to_file=save_end_events, 
    save_dir = os.path.join(save_dir_base, 'EndEvents'), 
    save_name=r'end_events.csv', 
    index=True
)
#-------------------------
print(f"save_dir_base = {save_dir_base}")
print('end_events_save_args')
for k,v in end_events_save_args.items():
    print(f"\t{k} : {v}")
#-------------------------
if save_dfs_to_file or save_end_events:
    if not os.path.exists(save_dir_base):
        os.makedirs(save_dir_base)
    #-----
    if save_end_events and not os.path.exists(end_events_save_args['save_dir']):
        os.makedirs(end_events_save_args['save_dir'])

In [ ]:
#----------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------
assert(save_dfs_to_file+read_dfs_from_file <=1) # Should never both read and write!
assert(pd.to_datetime(date_1)-pd.to_datetime(date_0) > min_window_width+buffer_time_left+buffer_time_rght)
#--------------------------------------------------
if not read_dfs_from_file:
    conn_outages = Utilities.get_utldb01p_oracle_connection()
    conn_aws = Utilities.get_athena_prod_aws_connection()

In [ ]:
# save_dfs_to_file   = False
# read_dfs_from_file = True

# ---------------------------------------------------------------
# OUTAGES
# ---------------------------------------------------------------

### build_or_load_df_outage_OG

In [ ]:
if read_dfs_from_file:
    df_outage_OG = pd.read_pickle(os.path.join(save_dir_base, 'df_outage_OG.pkl'))
    csv_cols_and_types_to_convert_dict = {'CI_NB':np.int32, 'CMI_NB':np.float64, 'OUTG_REC_NB':[np.float64, np.int32]}
    df_outage_OG = Utilities_df.convert_col_types(df_outage_OG, csv_cols_and_types_to_convert_dict)
else:
    sql_outage_full = DOVSOutages_SQL.build_sql_std_outage(
        mjr_mnr_cause   = None, 
        include_premise = True, 
        date_range      = [date_0, date_1], 
        states          = states, 
        opcos           = opcos, 
        cities          = cities
    ).get_sql_statement()
    print(sql_outage_full)
    #-------------------------
    df_outage_OG = pd.read_sql_query(
        sql_outage_full, 
        conn_outages, 
        dtype = {
            'CI_NB':np.int32, 
            'CMI_NB':np.float64, 
            'OUTG_REC_NB':np.int32
        }
    )
    csv_cols_and_types_to_convert_dict = {'CI_NB':np.int32, 'CMI_NB':np.float64, 'OUTG_REC_NB':[np.float64, np.int32]}
    df_outage_OG = Utilities_df.convert_col_types(df_outage_OG, csv_cols_and_types_to_convert_dict)
    #-------------------------
    if save_dfs_to_file:
        df_outage_OG.to_pickle(os.path.join(save_dir_base, 'df_outage_OG.pkl'))
#-------------------------
print(f'df_outage_OG.shape = {df_outage_OG.shape}')
# df_outage = df_outage_OG.copy()

In [ ]:
df_outage = df_outage_OG.copy()

# Find all outages between date_0 and date_1 for PNs in df_outage

# Need to find all PNs, which consist not only of those directly from df_outage, but also those not in df_outage who were connected to transformers having entries in df_outage

### build_or_load_mp_df_xfmrs

In [ ]:
if read_dfs_from_file:
    mp_df_PNs   = pd.read_pickle(os.path.join(save_dir_base, 'mp_df_PNs_no_outg.pkl'))
    mp_df_xfmrs = pd.read_pickle(os.path.join(save_dir_base, 'mp_df_xfmrs_no_outg.pkl'))
else:
    conn_aws = Utilities.get_athena_prod_aws_connection()
    PNs = df_outage['PREMISE_NB'].unique().tolist()
    #-------------------------
    start=time.time()
    mp_df_PNs, sql_stmnts = MeterPremise.get_distinct_trsf_pole_nbs_for_PNs(
        PNs        = PNs, 
        batch_size = 10000, 
        conn_aws   = conn_aws, 
        return_sql = True, 
        states     = states, 
        opcos      = opcos, 
        cities     = cities
    )
    print(f"mp_df_PNs: {time.time()-start}")
    #-------------------------
    start=time.time()
    mp_xfmrs = GenAn(
        df_construct_type         = DFConstructType.kRunSqlQuery, 
        contstruct_df_args        = dict(conn_db=Utilities.get_athena_prod_aws_connection()), 
        init_df_in_constructor    = True, 
        build_sql_function        = MeterPremise.build_sql_meter_premise, 
        build_sql_function_kwargs = dict(
            cols_of_interest = ['trsf_pole_nb', 'prem_nb', 'mfr_devc_ser_nbr', 'inst_ts', 'rmvl_ts', 'state_cd'], 
            trsf_pole_nb     = [x for x in mp_df_PNs['trsf_pole_nb'].unique() if x!='TRANSMISSION'], 
            field_to_split   = 'trsf_pole_nb', 
            states           = states, 
            opcos            = opcos, 
            cities           = cities
        )
    )
    print(f"mp_df_xfmrs: {time.time()-start}")
    mp_df_xfmrs = mp_xfmrs.df.copy()
    #-------------------------
    if save_dfs_to_file:
        mp_df_PNs.to_pickle(os.path.join(save_dir_base, 'mp_df_PNs_no_outg.pkl'))
        mp_df_xfmrs.to_pickle(os.path.join(save_dir_base, 'mp_df_xfmrs_no_outg.pkl'))
#---------------------------------------------------------------------------
# PNs = df_outage['PREMISE_NB'].unique().tolist()
PNs = list(set(df_outage['PREMISE_NB'].unique().tolist() + mp_df_xfmrs['prem_nb'].unique().tolist()))

# Build zip code information for transformers, and possibly restrict data to single zipcode transformers only

### build_or_load_trsf_pole_zips_info

In [ ]:
if read_dfs_from_file:
    mp_for_zips_df    = pd.read_pickle(os.path.join(save_dir_base, 'mp_for_zips_df.pkl'))
    trsf_pole_df_full = pd.read_pickle(os.path.join(save_dir_base, 'trsf_location_info_df.pkl'))
    trsf_pole_zips_df = pd.read_pickle(os.path.join(save_dir_base, 'trsf_pole_zips_df.pkl'))
else:
    zips_dict = OutageDAQ.build_trsf_pole_zips_df(
        field_to_split_and_val = ('premise_nbs', PNs), 
        states                 = states, 
        opcos                  = opcos, 
        cities                 = cities
    )
    #----------
    trsf_pole_zips_df = zips_dict['trsf_pole_zips_df']
    trsf_pole_df_full = zips_dict['trsf_pole_df_full']
    mp_for_zips_df    = zips_dict['mp_for_zips_df']
    #--------------------------------------------------
    if save_dfs_to_file:
        mp_for_zips_df.to_pickle(   os.path.join(save_dir_base, 'mp_for_zips_df.pkl'))
        trsf_pole_df_full.to_pickle(os.path.join(save_dir_base, 'trsf_location_info_df.pkl'))
        trsf_pole_zips_df.to_pickle(os.path.join(save_dir_base, 'trsf_pole_zips_df.pkl'))
#--------------------------------------------------
if single_zip_xfmrs_only:
    trsf_pole_nzips   = trsf_pole_zips_df.drop(columns=['zip+4']).drop_duplicates()['trsf_pole_nb'].value_counts()
    single_zip_poles  = trsf_pole_nzips[trsf_pole_nzips==1].index.tolist()
    #-----
    trsf_pole_zips_df = trsf_pole_zips_df[trsf_pole_zips_df['trsf_pole_nb'].isin(single_zip_poles)]
    #-----
    PNs = mp_for_zips_df[mp_for_zips_df['trsf_pole_nb'].isin(single_zip_poles)]['prem_nb'].unique().tolist()

### build_or_load_all_outages_df

In [ ]:
# Now, for all premise numbers in df_outage, we want to find all outages suffered between date_0 and date_1
# There are two methods for achieving this, using either method='query_pns_only' or method='query_all' in find_all_outages_for_pns
# NOTE: It is possible that all needed info already contained in df_outage! 
#       But that is reliant upon the user having everything set up perfectly....so safest just to build...
verbose    = True
n_update   = 10
batch_size = 1000

method = 'decide_at_runtime'
# addtnl_build_sql_std_outage_kwargs=None
addtnl_build_sql_std_outage_kwargs=dict(
    states = states, 
    opcos  = opcos, 
    cities = cities
)

In [ ]:
len(PNs)

In [ ]:
#-------------------------*************************-------------------------*************************
start = time.time()
all_outages_df = OutageDAQOtBL.find_all_outages_for_pns(
    PNs                                = PNs, 
    date_0                             = date_0, 
    date_1                             = date_1, 
    cols_of_interest                   = None, 
    mjr_mnr_cause                      = None, 
    method                             = method, 
    addtnl_build_sql_std_outage_kwargs = addtnl_build_sql_std_outage_kwargs, 
    verbose                            = verbose, 
    n_update                           = n_update, 
    batch_size                         = batch_size
)
print(f"Time to run OutageDAQOtBL.find_all_outages_for_pns: {time.time()-start}")
print(f"# Unique PNs in df_outage:      {df_outage['PREMISE_NB'].nunique()}")
print(f"# Unique PNs in all_outages_df: {all_outages_df['PREMISE_NB'].nunique()}")
print(time.time()-start)

In [ ]:
# save_dfs_to_file   = True
# read_dfs_from_file = False

In [ ]:
#-------------------------*************************-------------------------*************************
# If grouping by transformer, the trsf_pole_nb from MeterPremise must be merged with all_outages_df
# Also, the active meters at the time of outage must be selected by comparing inst_ts,rmvl_ts to 
#   DT_OFF_TS_FULL,DT_ON_TS.
# This is documented in the code below
#-------------------------
if groupby_col=='trsf_pole_nb':
    if read_dfs_from_file:
        df_mp = pd.read_pickle(os.path.join(save_dir_base, 'df_mp_no_outg.pkl'))
        #-----
        # Check for df_mp_outg_rec_nb_col in df_mp, regardless of case
        #   If contained, must be dropped and duplicates removed
        #   *See comment above df_mp_outg_rec_nb_col initial assignment
        df_mp = Utilities_df.drop_col_case_insensitive(
            df        = df_mp, 
            col       = df_mp_outg_rec_nb_col, 
            inplace   = True, 
            drop_dups = True
        )
    else:
        df_mp = MeterPremise.build_mp_df_curr_hist_for_PNs(
            PNs                    =  PNs, 
            mp_df_curr             = None,
            mp_df_hist             = None, 
            join_curr_hist         = True, 
            addtnl_mp_df_curr_cols = None, 
            addtnl_mp_df_hist_cols = None, 
            assert_all_PNs_found   = False, 
            assume_one_xfmr_per_PN = True, 
            drop_approx_duplicates = True
        )
        if save_dfs_to_file:
            df_mp.to_pickle(os.path.join(save_dir_base, 'df_mp_no_outg.pkl'))
    #--------------------------------------------------
    # Some premise numbers from DOVS are missing from df_mp.
    # This is not an issue with the code, I checked. 
    # This means DOVS says a premise was affected by an outage, but at the time of the outage there were 
    #   no active meters on the premise.
    # My question is: How did DOVS therefore know the premise was affected?
    # How are premise numbers in DOVS determined?
    #-------------------------
    # I want to at least get a count to quantify the situation described above, i.e., how many premises from DOVS
    #   did not have any active meters at the time of the outage.
    # Note, for this, I cannot simply do, e.g., 
    #     set(all_outages_df['PREMISE_NB'].unique()).difference(set(df_mp['prem_nb'].unique()))
    #   as this might reflect a smaller number of missing PNs than in reality, as df_mp has not yet been chopped down
    #   to only those present at time of outage (which is done below comparing 'inst_ts' to 'DT_OFF_TS_FULL' and 
    #   'rmvl_ts' to 'DT_ON_TS')
    #-------------------------
    # The meters present at the time of the outages can only be select after all_outages_df and df_mp are merged.
    #-------------------------
    # Note: A left merge is used below instead of an inner to protect against the case of a df_mp (being read in from a CSV 
    #       file) which contains extra entries than in all_outages_df
    #-------------------------
    all_outages_df = DOVSOutages.merge_df_outage_with_mp(
        df_outage          = all_outages_df, 
        df_mp              = df_mp,  
        merge_on_outg      = ['PREMISE_NB'], 
        merge_on_mp        = ['prem_nb'], 
        cols_to_include_mp = None, 
        drop_cols          = None, 
        rename_cols        = None, 
        how                = 'left', 
        inplace            = True
    )

    #-------------------------
    # Only include serial numbers which were present at the time of the outage.
    #-----
    # NOTE the use of .fillna(pd.Timestamp.min) (YES, MIN) below, as this is different from MeterPremise.get_active_SNs_for_PNs_at_datetime_interval
    #   and MeterPremise.merge_df_with_active_mp.
    # This is needed so the premises missing from df_mp are not removed at this stage (yes, they will ultimately be removed, 
    #   but I don't want them removed yet because I want to track them!)
    # Without this, any entry with 'inst_ts'=NaT would be removed, as a comparison of NaT to anything returns False
    #-----
    all_outages_df = Utilities_df.convert_col_types(
        df                  = all_outages_df, 
        cols_and_types_dict = {
            'inst_ts' : datetime.datetime, 
            'rmvl_ts' : datetime.datetime
        }, 
        to_numeric_errors   = 'coerce', 
        inplace             = True
    )
    #-----
    all_outages_df = all_outages_df[
        (all_outages_df['inst_ts'].fillna(pd.Timestamp.min) <= all_outages_df['DT_OFF_TS_FULL']) & 
        (all_outages_df['rmvl_ts'].fillna(pd.Timestamp.max) >  all_outages_df['DT_ON_TS'])
    ]

    #-------------------------
    # Find the entries with missing df_mp data, i.e., find the entries where DOVS says a premise was affected by an outage, 
    #   but at the time of the outage there were no active meters on the premise.
    all_outages_df_for_non_active_pns = all_outages_df[all_outages_df[df_mp.columns].isna().all(axis=1)].copy()
    non_active_pns_from_DOVS          = all_outages_df_for_non_active_pns['PREMISE_NB'].unique().tolist()

    # And remove the entries with missing df_mp data from all_outages_df
    all_outages_df = all_outages_df.dropna(subset=df_mp.columns, how='all')
    #-------------------------
    print("""
    Some premise numbers from DOVS are missing from df_mp
    This is not an issue with the code, I checked.  
    This means DOVS says a premise was affected by an outage, but at the time of the outage there were no active meters on the premise.
    My question is: How did DOVS therefore know the premise was affected?
    How are premise numbers in DOVS determined?
    """)
    print(f"Number of premise numbers from DOVS without an active meter at outage time: {len(non_active_pns_from_DOVS)}") 
    #-------------------------
    # At this point, any trsf_pole_nbs to be excluded can be removed
    # Remove 'TRANSMISSION', 'PRIMARY', and 'NETWORK' transformers
    all_outages_df = all_outages_df[~all_outages_df['trsf_pole_nb'].isin(['TRANSMISSION', 'NETWORK', 'PRIMARY'])] 
    #--------------------------------------------------

### build_or_load_df_no_outage

In [ ]:
#-------------------------*************************-------------------------*************************
# Find the clean windows for each group and build df_no_outage
#-------------------------
start = time.time()
clean_windows_by_grp = OutageDAQOtBL.find_clean_windows(
    df                     = all_outages_df, 
    groupby_col            = groupby_col, 
    min_window_width       = min_window_width, 
    buffer_time_left       = buffer_time_left, 
    buffer_time_rght       = buffer_time_rght, 
    date_1                 = date_1, 
    set_search_window      = True, 
    pd_selection_stategy   = pd_selection_stategy, 
    search_window_strategy = search_window_strategy, 
    outg_beg_col           = 'DT_OFF_TS_FULL', 
    outg_end_col           = 'DT_ON_TS'
)
print(time.time()-start)
#-------------------------
# All groups (trsf_pole_nbs, PREMISE_NBs, etc.) in clean_windows_by_grp should also be found in all_outages_df, 
#   but the reverse is not true
assert(len(set(clean_windows_by_grp[groupby_col].unique()).difference(set(all_outages_df[groupby_col].unique())))==0)

# Groups where no clean time period was found
grps_with_no_clean = all_outages_df[~all_outages_df[groupby_col].isin(clean_windows_by_grp[groupby_col].unique())]

print(f"groupby_col = {groupby_col}")
print(f"a. # Groups:                      {all_outages_df[groupby_col].nunique()}")
print(f"b. # Groups with clean period:    {clean_windows_by_grp[groupby_col].nunique()}")
print(f"c. # Groups without clean period: {len(set(all_outages_df[groupby_col].unique()).difference(set(clean_windows_by_grp[groupby_col].unique())))}")
print("NOTE: There may be a difference of 1 between a and b+c due to fact that nunique() does not including NaNs but unique does")
#--------------------------------------------------
if save_dfs_to_file:
    all_outages_df.to_pickle(os.path.join(save_dir_base, 'all_outages_df.pkl'))
    clean_windows_by_grp.to_pickle(os.path.join(save_dir_base, 'clean_windows_by_grp.pkl'))

In [ ]:
# all_outages_df=pd.read_pickle(os.path.join(save_dir_base, 'all_outages_df.pkl'))
# clean_windows_by_grp=pd.read_pickle(os.path.join(save_dir_base, 'clean_windows_by_grp.pkl'))
# df_no_outage=pd.read_pickle(os.path.join(save_dir_base, 'df_no_outage.pkl'))
# df_mp = pd.read_pickle(os.path.join(save_dir_base, 'df_mp_no_outg.pkl'))

In [ ]:
#-------------------------*************************-------------------------*************************
# Merge clean_windows_by_grp with df_mp
#-------------------------
clean_windows_by_grp_mrg_mp = pd.merge(
    clean_windows_by_grp, 
    df_mp, 
    left_on='trsf_pole_nb', 
    right_on='trsf_pole_nb', 
    how='left'
)
clean_windows_by_grp_mrg_mp = Utilities_df.convert_col_types(
    df                  = clean_windows_by_grp_mrg_mp, 
    cols_and_types_dict = {
        'inst_ts' : datetime.datetime, 
        'rmvl_ts' : datetime.datetime
    }, 
    to_numeric_errors   = 'coerce', 
    inplace             = True
)


clean_windows_by_grp_mrg_mp = clean_windows_by_grp_mrg_mp[
    (clean_windows_by_grp_mrg_mp['inst_ts'].fillna(pd.Timestamp.min) <= clean_windows_by_grp_mrg_mp['t_search_min']) & 
    (clean_windows_by_grp_mrg_mp['rmvl_ts'].fillna(pd.Timestamp.max)  > clean_windows_by_grp_mrg_mp['t_search_max'])
]
#--------------------------------------------------
if save_dfs_to_file:
    clean_windows_by_grp_mrg_mp.to_pickle(os.path.join(save_dir_base, 'clean_windows_by_grp_mrg_mp.pkl'))
#--------------------------------------------------
df_no_outage = clean_windows_by_grp_mrg_mp.copy()
df_no_outage = df_no_outage.sort_values(by=[groupby_col, 'prem_nb', 't_search_min'], ignore_index=True)
#--------------------------------------------------
# Add no_outg_rec_nb column to allow easier grouping when building rcpo_dfs
rand_pfx                       = Utilities.generate_random_string(str_len=5, letters=string.ascii_letters + string.digits)
df_no_outage['no_outg_rec_nb'] = df_no_outage.groupby(['trsf_pole_nb', 't_search_min', 't_search_max']).ngroup()
df_no_outage['no_outg_rec_nb'] = rand_pfx + df_no_outage['no_outg_rec_nb'].astype(str)
#--------------------------------------------------
if save_dfs_to_file:
    df_no_outage.to_pickle(os.path.join(save_dir_base, 'df_no_outage_FINAL.pkl'))

### build_or_load_df_no_outage_slim

In [ ]:
#-------------------------*************************-------------------------*************************
# Convert df_no_outage to consolidated (slim) verion if run_using_slim
#-------------------------
if run_using_slim:
    #--------------------------------------------------
    # Convert to slim 
    cols_shared_by_group     = None
    cols_to_collect_in_lists = ['prem_nb']
    rename_cols              = {'prem_nb':'premise_nbs'}
    if groupby_col=='trsf_pole_nb':
        cols_to_collect_in_lists.append('mfr_devc_ser_nbr')
        rename_cols['mfr_devc_ser_nbr'] = 'serial_numbers'
    #-------------------------
    if groupby_col=='trsf_pole_nb':
        consol_groupby_cols = ['no_outg_rec_nb', groupby_col, 't_search_min', 't_search_max']
    elif groupby_col=='PREMISE_NB':
        consol_groupby_cols =[ 'no_outg_rec_nb', 't_search_min', 't_search_max']
    else:
        assert(0)
    #-------------------------
    if search_window_strategy=='all_subwindows':
        consol_groupby_cols.append('is_first_after_outg')
    #-------------------------
    df_no_outage_slim = Utilities_df.consolidate_df(
        df                       = df_no_outage, 
        groupby_cols             = consol_groupby_cols, 
        cols_shared_by_group     = cols_shared_by_group, 
        cols_to_collect_in_lists = cols_to_collect_in_lists, 
        rename_cols              = rename_cols, 
        verbose                  = True
    )
    #-------------------------
    df_no_outage_slim=df_no_outage_slim.reset_index()
    if groupby_col=='trsf_pole_nb':
        df_no_outage_slim = df_no_outage_slim.set_index(['no_outg_rec_nb', 'trsf_pole_nb'], drop=False)
        df_no_outage_slim.index.names=['idx_no_outg_rec_nb', 'idx_trsf_pole_nb']
    #-------------------------
    df_no_outage_slim['premise_nbs'] = df_no_outage_slim['premise_nbs'].apply(sorted)
    if groupby_col=='trsf_pole_nb':
        df_no_outage_slim['serial_numbers'] = df_no_outage_slim['serial_numbers'].apply(sorted)
    #-------------------------
    if save_dfs_to_file:
        df_no_outage_slim.to_pickle(os.path.join(save_dir_base, 'df_no_outage_slim.pkl'))

# Collect events

In [ ]:
# df_no_outage = pd.read_pickle(os.path.join(save_dir_base, 'df_no_outage_FINAL.pkl'))

### collect_events

In [ ]:
#----------
df_construct_type             = DFConstructType.kRunSqlQuery
contstruct_df_args_end_events = None



cols_of_interest_end_dev_event = TableInfos.AMIEndEvents_TI.std_columns_of_interest
verbose  = True
n_update = 1

In [ ]:
if groupby_col=='trsf_pole_nb':
    addtnl_groupby_cols = ['trsf_pole_nb', 'no_outg_rec_nb']
if groupby_col=='PREMISE_NB':
    addtnl_groupby_cols = ['no_outg_rec_nb']
#-------------------------
if search_window_strategy=='all_subwindows':
    addtnl_groupby_cols.append('is_first_after_outg')

In [ ]:
#--------------------------------------------------
end_events_sql_function_kwargs = dict(
    cols_of_interest                  = cols_of_interest_end_dev_event, 
    join_mp_args                      = False, 
    df_args                           = dict(
        addtnl_groupby_cols = addtnl_groupby_cols, 
        t_search_min_col    = 't_search_min', 
        t_search_max_col    = 't_search_max'
    ), 
    field_to_split                    = 'df_mp_no_outg', 
    field_to_split_location_in_kwargs = ['df_mp_no_outg'], 
    sort_coll_to_split                = False,
    verbose                           = verbose, 
    n_update                          = n_update
)
#----------
# if opcos is not None:
addtnl_end_events_sql_function_kwargs = dict(
    build_sql_function_kwargs = dict(
        states = states, 
        opcos  = opcos, 
        cities = cities
    )
)
end_events_sql_function_kwargs = {
    **end_events_sql_function_kwargs, 
    **addtnl_end_events_sql_function_kwargs
}
#--------------------------------------------------
if run_using_slim:
    batch_size = 10
    #----------
    end_events_sql_function_kwargs = Utilities.supplement_dict_with_default_values(
        to_supplmnt_dict    = end_events_sql_function_kwargs, 
        default_values_dict = dict(
            df_mp_no_outg = df_no_outage_slim, 
            batch_size    = batch_size, 
            df_args       = dict(
                mapping_to_ami     = DfToSqlMap(df_col='premise_nbs', kwarg='premise_nbs', sql_col='aep_premise_nb'), 
                is_df_consolidated = True
            )
        ), 
        extend_any_lists    = True,
        inplace             = True
    )
#-------------------------
else:
    batch_size=200
    #----------
    if groupby_col=='trsf_pole_nb':
        df_no_outage=df_no_outage.sort_values(by=['no_outg_rec_nb', 'trsf_pole_nb', 'prem_nb', 't_search_min'], ignore_index=True)
    if groupby_col=='PREMISE_NB':
        df_no_outage=df_no_outage.sort_values(by=['no_outg_rec_nb', 'PREMISE_NB', 't_search_min'], ignore_index=True)
    #----------
    end_events_sql_function_kwargs = Utilities.supplement_dict_with_default_values(
        to_supplmnt_dict = end_events_sql_function_kwargs, 
        default_values_dict = dict(
            df_mp_no_outg = df_no_outage, 
            batch_size    = batch_size, 
            df_args       = dict(
                mapping_to_ami     = DfToSqlMap(df_col='prem_nb', kwarg='premise_nbs', sql_col='aep_premise_nb'), 
                is_df_consolidated = False
            )
        ), 
        extend_any_lists = True,
        inplace          = True
    )

In [ ]:
start=time.time()

exit_status = Utilities.run_tryexceptwhile_process(
    func                = AMIEndEvents,
    func_args_dict      = dict(
        df_construct_type         = df_construct_type, 
        contstruct_df_args        = contstruct_df_args_end_events, 
        build_sql_function        = AMIEndEvents_SQL.build_sql_end_events_for_no_outages, 
        build_sql_function_kwargs = end_events_sql_function_kwargs, 
        init_df_in_constructor    = True, 
        save_args                 = end_events_save_args
    ), 
    max_calls_per_min   = 1, 
    lookback_period_min = 15, 
    max_calls_absolute  = 1000, 
    verbose             = True
)
print(f'exit_status = {exit_status}')

build_time = time.time()-start
print(build_time)

In [ ]:
assert(0)